In [ ]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib import pyplot
import rasterio.mask
import fiona
import json

In [ ]:
OUTPUT_FILE_PATH = "assets/gibf.parquet"

In [ ]:
# Ocurrence species data -> OSD_df
OSD_df = pd.read_parquet(OUTPUT_FILE_PATH)

# Creating Geometry
OSD_df['geometry'] = list(zip(OSD_df["decimalLongitude"], OSD_df["decimalLatitude"]))
OSD_df['geometry'] = OSD_df["geometry"].apply(Point)

# Create the geodataframe
OSD_geoframe = gpd.GeoDataFrame(
    OSD_df,
    crs = {'init': 'epsg:4326'},
    geometry = OSD_df['geometry']
)
OSD_geoframe = OSD_geoframe.to_crs("EPSG:4326")
OSD_geoframe.reset_index(drop=True, inplace = True)

coord_list = [(x,y) for x,y in zip(OSD_geoframe['geometry'].x , OSD_geoframe['geometry'].y)]

OSD_geoframe['Year'] = OSD_geoframe.eventDate.dt.year
OSD_df = pd.DataFrame(OSD_geoframe)

In [ ]:
print(OSD_geoframe['decimalLatitude'].mean(), OSD_geoframe['decimalLongitude'].mean())

In [ ]:
fig = px.scatter_mapbox(
    OSD_geoframe.sort_values(by='Year',ascending=True),
    lat="decimalLatitude",
    lon="decimalLongitude",
    hover_name="stateProvince",
    hover_data=["individualCount"],
    color_discrete_sequence=["fuchsia"],
    zoom=3,
    height=450,
    animation_frame="Year",
    opacity=1
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
#fig.update_layout(mapbox_bounds={"west": -90, "east": 12, "south": -40, "north": 0})
fig.show()

In [ ]:
fig = px.scatter_mapbox(
    OSD_geoframe.sort_values(by='Year',ascending=True),
    lat="decimalLatitude",
    lon="decimalLongitude",
    hover_name="stateProvince",
    hover_data=["individualCount"],
    color_discrete_sequence=["fuchsia"],
    zoom=3,
    height=500,
    opacity=1
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
OSD_geoframe.head()

## Data description

Collection Code

In [ ]:
individual_count_per_collection_code = OSD_df[['individualCount','collectionCode']].groupby(['collectionCode']).sum().sort_values(by='individualCount')

In [ ]:
individual_count_per_collection_code = OSD_df[['individualCount','collectionCode']].groupby(['collectionCode']).sum().sort_values(by='individualCount')
fig = px.bar(individual_count_per_collection_code, x='individualCount', y=individual_count_per_collection_code.index, title="Individual count per collection", orientation='h')
fig.show()

## Creating GeoJSON File

In [ ]:
# Source: IBGE:  bcim_2016_21_11_2018
INFOS_UFS = gpd.read_file("assets/bcim_2016_21_11_2018.gpkg", layer = 'lim_unidade_federacao_a')[['sigla','geometry']].rename(columns={'sigla':'stateProvince'})

In [ ]:
INFOS_UFS.to_file("assets/UFS_JSON", driver = "GeoJSON")
with open("assets/UFS_JSON") as geofile:
    j_file = json.load(geofile)

In [ ]:
OSD_df.head()

In [ ]:
count_per_state = OSD_df[['stateProvince','individualCount']].groupby(['stateProvince']).sum()

In [ ]:
INFOS_UFS_COUNT_PER_STATE = INFOS_UFS.merge(count_per_state, left_on='stateProvince', right_index=True, how='left').fillna(0)

In [ ]:
INFOS_UFS_COUNT_PER_STATE

In [ ]:
from urllib import request

url = 'https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/brazil-states.geojson'
with request.urlopen(url) as f:
    brazil_states = json.load(f)

In [ ]:
fig = px.choropleth_mapbox(
        INFOS_UFS_COUNT_PER_STATE,
        locations="stateProvince",
        center={"lat": -16.95, "lon": -47.78},
        geojson=brazil_states,
        featureidkey='properties.sigla', # add
        color_continuous_scale="Redor",
        opacity=0.5,
        zoom=3,
        color="individualCount",
        hover_data={"stateProvince": True},
        mapbox_style='carto-darkmatter'
)

fig.show()

## Overlay temperature

In [ ]:
type(brazil_states)

In [ ]:
# List of coordinates
coord_list = [(x,y) for x,y in zip(OSD_geoframe['geometry'].x , OSD_geoframe['geometry'].y)]
print(coord_list[0:5])

In [ ]:
len(coord_list)

In [ ]:
brazil_states['type']

In [ ]:
len(brazil_states['features'][0]['geometry']['coordinates'][0][0])

In [ ]:
coord_list = []
for brazil_state in brazil_states['features']:
    coord_list = coord_list + brazil_state['geometry']['coordinates'][0][0]

In [ ]:
# Creating Geometry
FEAT_DATAFRAME = pd.DataFrame()

FEAT_DATAFRAME['geometry'] = coord_list

In [ ]:
FEAT_DATAFRAME['geometry'] = FEAT_DATAFRAME["geometry"].apply(Point)

In [ ]:
from pathlib import Path

In [ ]:
for f in raster_features:
    src = rasterio.open(f)
    FEAT_DATAFRAME[Path(f).stem] = [x for x in src.sample(coord_list)]
    FEAT_DATAFRAME[Path(f).stem] = FEAT_DATAFRAME[Path(f).stem].astype('float64')
train_vec = FEAT_DATAFRAME.copy()
train_vec.head()

In [ ]:
train_vec['decimalLatitude'] = [i.y for i in train_vec['geometry']]
train_vec['decimalLongitude'] = [i.x for i in train_vec['geometry']]

In [ ]:
import rasterio
import glob

In [ ]:
raster_features = sorted(glob.glob(
    'assets/wc2.1_2.5m_tavg/*.tif'))
# check number of features 
print('\nThere are', len(raster_features), 'raster features.')

In [ ]:
train_vec

In [ ]:
with fiona.open("BR_UF_2022\BR_UF_2022.shp", "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]
    
with rasterio.open(raster_features[0]) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

In [ ]:
raster_features[0]

In [ ]:
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open("wc2.1_2.5m_tavg_01.tif", "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
from rasterio.plot import show

In [ ]:
src = rasterio.open("wc2.1_2.5m_tavg_01.tif")


In [ ]:
arr = src.read()

In [ ]:
arr[0][0]

# Testing Features

In [ ]:
import numpy as np
import plotly.graph_objs as go
from shapely.geometry import box, mapping

In [ ]:
# Read the GeoTIFF file
with rasterio.open("wc2.1_2.5m_tavg_01.tif") as src:
    raster_data = src.read(1)  # Read the first band
    extent = src.bounds  # Get the spatial extent of the raster

# Filter out negative values
raster_data_non_negative = np.where(raster_data < 0, np.nan, raster_data)
# Create a GeoJSON Polygon representing the extent
extent_polygon = box(extent.left, extent.bottom, extent.right, extent.top)
extent_geojson = mapping(extent_polygon)

In [ ]:
raster_data.shape

In [ ]:
extent_geojson

In [ ]:
# Read the GeoTIFF file
with rasterio.open("wc2.1_2.5m_tavg_01.tif") as src:
    raster_data = src.read(1)  # Read the first band
    extent = src.bounds  # Get the spatial extent of the raster

# Filter out negative values
raster_data_non_negative = np.where(raster_data < 0, np.nan, raster_data)
# Create a GeoJSON Polygon representing the extent
extent_polygon = box(extent.left, extent.bottom, extent.right, extent.top)
extent_geojson = mapping(extent_polygon)

# Define the layout for the map
layout = go.Layout(
    title="Raster Plot (Non-negative values) with Mapbox",
    mapbox=dict(
        style="open-street-map",  # Choose the base map style
        center=dict(lon=(extent.left + extent.right) / 2, lat=(extent.top + extent.bottom) / 2),  # Set the map center
        zoom=10  # Set the initial zoom level
    ),
)

# Create the choropleth_mapbox trace
choropleth_mapbox = go.Choroplethmapbox(
    z=raster_data_non_negative,
    geojson=extent_geojson,
    locations=[[extent.left, extent.bottom], [extent.right, extent.top]],
    featureidkey="extent",
    colorscale="Viridis",
    colorbar=dict(title="Raster Values"),
)

# Create the figure
fig = go.Figure(data=choropleth_mapbox, layout=layout)

# Show the plot
fig.show()

In [ ]:

# Read the GeoTIFF file
with rasterio.open("wc2.1_2.5m_tavg_01.tif") as src:
    raster_data = src.read(1)  # Read the first band

# Filter out negative values
raster_data_non_negative = np.where(raster_data < 0, np.nan, raster_data)

# Create a Plotly heatmap trace
heatmap = go.Heatmap(z=raster_data_non_negative, colorscale="Viridis", showscale=True)

# Create the layout for the plot
layout = go.Layout(
    title="Raster Plot (Non-negative values)",
    xaxis=dict(title="Longitude"),
    yaxis=dict(title="Latitude"),
    width=800,
    height=600,
    margin=dict(l=40, r=40, t=80, b=40)
)

# Create the Plotly figure
fig = go.Figure(data=[heatmap], layout=layout)

# Display the plot
fig.show()


In [ ]:
raster_data.shape